In [ ]:
import mercury as mr
import numpy as np
# contracts
from qablet_contracts.rate.swaption import swaption_timetable, bermuda_swaption_timetable
from qablet_contracts.bond.zero import zcb_timetable, zbp_timetable
from qablet_contracts.bond.fixed import fixed_bond_timetable
# models
from qablet.hullwhite.fd import HWFDModel
from qablet.base.fixed import FixedModel

In [ ]:
# Create Mercury App
app = mr.App(
    title="Qablet: Rate Instruments",
    description="Samples app in Mercury",
    allow_download=False,
)

mercury.App

In [ ]:
# Select Instrument type and create timetable
inst_type = mr.Select(label="Type of Instrument", value="Zero Bond",
                      choices=["Zero Bond", "Zero Bond Put", "Fixed Bond", "Vanilla Swaption", "Bermuda Swaption" ]).value

_ = mr.Note(text="**Instrument Terms**")

if inst_type == "Zero Bond":
    maturity = mr.Select(label="Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    timetable = zcb_timetable("USD", float(maturity))

elif inst_type == "Zero Bond Put":
    opt_maturity_str = mr.Select(label="Option Maturity (Years)",
                            value="1",
                            choices=["1", "2", "5"]).value
    opt_maturity = int(opt_maturity_str)
    maturity_val = str(opt_maturity + 1)
    maturity_choices=[str(x + opt_maturity) for x in [1, 2, 5]]
    bond_maturity = mr.Select(label="Bond Maturity (Years)",
                            value=maturity_val,
                            choices=maturity_choices).value
    strike = mr.Slider(label="Strike (%)", value=95, min=10, max=200).value / 100
    timetable = zbp_timetable("USD", float(opt_maturity), float(bond_maturity), strike)

elif inst_type == "Fixed Bond":
    freq = mr.Select(label="Payments per year", value="4", choices=["1", "2", "4"]).value
    fixed_rate = mr.Slider(label="Fixed Rate (%)", value=3, min=0, max=10).value / 100
    maturity = mr.Select(label="Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    timetable = fixed_bond_timetable("USD", fixed_rate, float(maturity), int(freq))

elif inst_type in ["Vanilla Swaption", "Bermuda Swaption"]:
    fixed_rate = mr.Slider(label="Fixed Rate (%)", value=3, min=0, max=10).value / 100
    start = mr.Slider(label="Start in (days)", value=30, min=0, max=180).value / 365
    maturity = mr.Select(label="Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    freq = mr.Select(label="Payments per year", value="4", choices=["1", "2", "4"]).value

    num_periods = int(maturity) * int(freq)
    periods = start + np.linspace(0, int(maturity), num = num_periods + 1)

    if inst_type == "Vanilla Swaption":
        timetable = swaption_timetable("USD", periods, strike_rate=fixed_rate)
    else:
        timetable = bermuda_swaption_timetable("USD", periods, strike_rate=fixed_rate)


mercury.Select

**Instrument Terms**

mercury.Select

## TimeTable

In [ ]:

print(timetable["events"].to_pandas()) # Display Timetable

  track  time op  quantity unit
0         2.0  +       1.0  USD


In [ ]:
# Select Model and model parameters
_ = mr.Note(text="**Model**")

# Choose model
model_type = mr.Select(label="Model", value="Hull-White FD",
                      choices=["Hull-White FD", "Fixed"]).value

**Model**

mercury.Select

In [ ]:
# Edit Discount rates (flat).
rate = mr.Slider(label="Discount Rate (%)", value=3, min=0, max=10).value / 100

times = np.array([0.0, 11])
rates = np.array([rate, rate])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

mercury.Slider

In [ ]:

if model_type == "Hull-White FD":
    model = HWFDModel()

    mean_rev = mr.Slider(label="Mean Reversion (%)", value=10, min=0, max=30).value / 100
    vol = mr.Slider(label="Rate Volatility(%)", value=3, min=0, max=30).value / 100
    dataset = {
        "BASE": "USD",
        "ASSETS": {"USD": discount_data},
        "FD": {
            "TIMESTEP": 1 / 500,
            "MAX_X": 0.10,
            "N_X": 1000,
        },
        "HW": {
            "MEANREV": mean_rev,
            "VOL": vol,
        },
    }
else:
    model = FixedModel()
    dataset = {
        "BASE": "USD",
        "ASSETS": {"USD": discount_data},
    }


mercury.Slider

mercury.Slider

In [ ]:
# Calculate Price
price, _ = model.price(timetable, dataset)
mr.NumberBox(data=round(price, 4), title="Price")
